# Summary

Here we test the tools developped in Utilities/Optobot_utils.py. We then use these tools to analyse Irene's optobot experiments.

# Package imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
sys.path.insert(0, "../Utilities")

# sys.path.append("/home/durrieu/Tracking_Analysis/Utilities")
# sys.path.append("/Users/ulric/git/Tracking_Analysis/Utilities")
import Utils
import Optobot_utils

import holoviews as hv
hv.extension('bokeh')

# Accessing the experiments

We first developp a function to find the experiments in the main directory. We then use a custom class to load the experiments, including metadata and DLC data.

In [ ]:
Exps = Optobot_utils.find_experiments(
    Utils.get_labserver() / "Experimental_data" / "Irene_Optobot"
)

#Exps

In [ ]:
TestFly = Exps[0]

In [ ]:
# Extract the npy file called experiment_dict.npy

# Load the experiment_dict.npy file
exp_dict = np.load(TestFly / "experiment_dict.npy", allow_pickle=True).item()

exp_dict

In [ ]:
exp_dict["fly0"]

In [ ]:
# Load the genotype_dict.npy file
gen_dict = np.load(TestFly.parent.parent / "genotype_dict.npy", allow_pickle=True).item()

gen_dict

In [ ]:
# Find a .pkl file in the folder
pkl_files = list(TestFly.glob("*.pkl"))

# Get the first .pkl file
pkl_file = pkl_files[0]

In [ ]:
# Load the .pkl file
import pickle
with open(pkl_file, "rb") as input:
    data = pickle.load(input)

In [ ]:
import pandas as pd

# Read the .pkl file
data = pd.read_pickle(pkl_file)

data

In [ ]:
# Solve multiindex

data.columns = data.columns.droplevel(0)

data.reset_index(inplace=True)

data

In [ ]:
print(data.columns)

In [ ]:
# Drop the top level of the multi-index for 'pos_x' and 'pos_y'
data.columns = (
    data.columns.droplevel(0)
    if "pos_x" in data.columns or "pos_y" in data.columns
    else data.columns
)

data

# Testing velocities

In [ ]:
TestFly = Optobot_utils.Fly(TestFly)

In [ ]:
TestFly.data

In [ ]:
dataset = TestFly.data

In [ ]:
TestFly.directory

In [ ]:
vel = TestFly.compute_velocity()

In [ ]:
vel

In [ ]:
# plot the velocity


hv.Curve(vel)

Looks like small movements below 0.1 mm/s could just be tracking noise. Let's use that to classify the frames as moving or not.

# Testing dataframe loading

In [ ]:
Exps[16]

In [ ]:
TestFly = Optobot_utils.Fly(Exps[16])

In [ ]:
len(Exps)

In [ ]:
TestFly.data

In [ ]:
Exps[0].parent.parent

# Try the whole dataset

In [ ]:
import pandas as pd

data_list = []

for i, exp in enumerate(Exps):
    try:
        # Load the data and add it to a list to be concatenated later
        data = Optobot_utils.Fly(exp).data
        # Add a unique identifier to each DataFrame
        data["id"] = i
        data_list.append(data)
    except:
        print(f"Failed to load {exp}")

In [ ]:
data_list[1]

In [ ]:
# Check for duplicates
duplicates = any(df.duplicated().any() for df in data_list)
if duplicates:
    print("Duplicate datasets found.")
else:
    print("No duplicate datasets found.")

In [ ]:

# Concatenate the DataFrames
dataset = pd.concat(data_list, ignore_index=True)

dataset

In [ ]:
try:
    dataset = pd.DataFrame()
    for i, df in enumerate(data_list):
        dataset = pd.concat([dataset, df], ignore_index=True)
except Exception as e:
    print(f"Error occurred while concatenating DataFrame {i}.")
    print(str(e))

In [ ]:
data_list[16]

In [ ]:
# Inspect the problematic DataFrame
problematic_df = data_list[16]

# Print the first few rows of the DataFrame
print(problematic_df.head())

# Print the index of the DataFrame
print(problematic_df.index)

# Check if the DataFrame has any duplicate indices
if problematic_df.index.duplicated().any():
    print("The DataFrame has duplicate indices.")
else:
    print("The DataFrame does not have duplicate indices.")

# Check if the DataFrame has any duplicate columns
if problematic_df.columns.duplicated().any():
    print("The DataFrame has duplicate columns.")
else:
    print("The DataFrame does not have duplicate columns.")

Testing nbstripout on semihg

In [ ]:
print("hello")

# All flies + velocities

In [ ]:
import pandas as pd

data_list = []

for i, exp in enumerate(Exps):
    try:
        # Load the data and add it to a list to be concatenated later
        data = Optobot_utils.Fly(exp).data
        # Add a unique identifier to each DataFrame
        data["id"] = i
        data_list.append(data)
    except:
        print(f"Failed to load {exp}")

# Concatenate the DataFrames
dataset = pd.concat(data_list, ignore_index=True)

dataset